In [4]:
import json

In [2]:
with open('../data/Wikihow_filtered_instructions.json', 'r') as f:
    instruct_answers = json.load(f)

with open('../data/Wikihow_time_filtered.json', 'r') as f:
    times = json.load(f)

with open('../data/tmp/Wikihow_sample_paired.json', 'r') as f:
    paired = json.load(f)

with open('../data/tmp/Wikihow_sample_tools_parse.json', 'r') as f:
    tools = json.load(f)

with open('../data/tmp/Wikihow_sample_tools_retrieved_parse.json', 'r') as f:
    other_tools = json.load(f)


In [14]:
target = 'How To Freeze Okra_0'
question = target.split('_')[0]
idx = target.split('_')[1]

print(question)
print('\n'.join(instruct_answers[question][idx]))
print('****')
print(times[question][idx])
print('****')
print(tools[target]['Tools'])
print('****')
print(tools[target]['Tool_steps'])
print('****')

pairs = paired[target][2]
question = pairs.split('_')[0]
idx = pairs.split('_')[1]

print(question)
print('\n'.join(instruct_answers[question][idx]))
print('****')
print(times[question][idx])
print('****')
print(other_tools[pairs]['Tools'])
print('****')
print(other_tools[pairs]['Tool_steps'])

How To Freeze Okra
1. Rinse the okra under cool running water.
2. Slice the stem off of each okra pod.
3. Blanch the okra in boiling water for 3-4 minutes.
4. Transfer the okra to a bowl of ice water for 3-4 minutes.
5. Air-dry the okra on a cutting board.
6. Slice the okra into bite-sized pieces.
7. Place the okra pieces on a baking sheet.
8. Freeze the okra on the baking sheet for 1 hour.
9. Transfer the frozen okra to a resealable container or plastic bag.
****
[2, 4, 3, 3, 5, 6, 2, 60, 2]
****
{'Knife': 1, 'Pot': 1, 'Bowl': 1, 'Cutting board': 1, 'Baking sheet': 1}
****
{'1': 'None', '2': 'Knife', '3': 'Pot', '4': 'Bowl', '5': 'Cutting board', '6': 'Knife', '7': 'Baking sheet', '8': 'None', '9': 'None'}
****
How To Freeze Cilantro
1. Rinse the cilantro in a bowl of cool water.
2. Pat the cilantro dry with a paper towel.
3. Blanch the cilantro by dipping it into boiling water for 15-30 seconds and then in ice-cold water for a few seconds.
4. Pinch off the leaves and discard the stem

In [54]:
with open('../data/GPT4o_sample_results.json', 'r') as f:
    gpt_results = json.load(f)

with open('../data/GPT4o_sample_results_with_tools.json', 'r') as f:
    gpt_results_with = json.load(f)

with open('../data/Wikihow_sample_best_scheduling.json', 'r') as f:
    best_results = json.load(f)

with open('../data/GPT4o_sample_with_tools_prompts.json', 'r') as f:
    prompts = json.load(f)

In [55]:
key = 'How To Clean Granite Tiles_0'
v_key = 'How To Clean Granite Countertops_1'

print(best_results[key][v_key]["makespan"] / 100)
print(gpt_results_with[key][v_key])
print(tools[key]['Tool_steps'])
print(other_tools[v_key]['Tool_steps'])
print(prompts[key][v_key])


17.0
A.1 (start at minutes 0, end at minutes 4) Tool: Dry mop or duster  
B.1 (start at minutes 4, end at minutes 9) Tool: None  
A.2 (start at minutes 9, end at minutes 15) Tool: None  
B.2 (start at minutes 15, end at minutes 22) Tool: Soft cloth  
A.3 (start at minutes 22, end at minutes 27) Tool: Soft cloth  

Total time: 27 minutes
{'1': 'Dry mop or duster', '2': 'None', '3': 'Soft cloth'}
{'1': 'None', '2': 'Soft cloth'}
You are an expert in solving procedural problems. I now have two procedural problems and give the steps to solve them and the time and tools required for each step. However, I only have valid tools, but unlimited materials. I need you to plan whether it is possible to complete these two tasks at the same time based on these valid tools, and give the best method.
[Question A:] 
How To Clean Granite Tiles
[Answer A:]
1. Wipe down the granite tile surface with a dry mop or duster to remove loose dirt or debris.(Time: 4 minutes) (Tools: Dry mop or duster)
2. Wipe dow

In [60]:
import re

cnt_fail = 0
cnt_not_best = 0
cnt_correct = 0
for key, value in gpt_results_with.items():
    for v_key, v_value in value.items():
        v_value_list = v_value.split('\n')
        for line in v_value_list:
            if ('Total time' in line) or ('Total Time' in line):
                minutes = line.split(':')[1].replace('minutes', '')
                match = re.search(r'[+-]?\d+(\.\d+)?', minutes)
                # print(key)
                # print(v_key)
                # print(minutes)
                try:
                    minutes = float(match.group())
                    # print(best_results[key][v_key]["makespan"] / 100)
                    if minutes > best_results[key][v_key]["makespan"] / 100:
                        cnt_not_best += 1
                    elif minutes == best_results[key][v_key]["makespan"] / 100:
                        cnt_correct += 1
                    else:
                        cnt_fail += 1
                except:
                    print('FAIL')
print(cnt_correct, cnt_not_best, cnt_fail)

FAIL
FAIL
FAIL
FAIL
1 22 5


In [1]:
import json

In [6]:
with open('../data/filtered_instructions/Wikihow_filtered_instructions.json', 'r') as f:
    instruct_answers = json.load(f)
with open('../data/match_tools/pair_tools_cars.json', 'r') as f:
    pair1 = json.load(f)
with open('../data/match_tools/pair_tools_pets.json', 'r') as f:
    pair2 = json.load(f)
with open('../data/time_step/Wikihow_time_GT10.json', 'r') as f:
    times = json.load(f)
tool_cnt = 0
sum_cnt = 0
instruct_cnt = 0
time_cnt = 0
conflict_tool = 0
instruct_dic = {}
tool_dic = {}
save_instruct = {}
save_tool = {}
for key, value in pair1.items():
    question = key.split('_')[0]
    idx = key.split('_')[1]
    for v_key, v_value in value.items():
        time_cnt += sum(times[question][idx])
        time_cnt += sum(times[v_key.split('_')[0]][v_key.split('_')[1]])
        tmp_set = set()
        for tmp_key, tmp_value in v_value[1].items():
            for x in tmp_value.split(','):
                if (x.strip(' ') != 'None'):
                    tmp_set.add(x.strip(' '))
        conflict_tool_set = set()
        for tmp_key, tmp_value in v_value[0].items():
            for x in tmp_value.split(','):
                if (x.strip(' ') in tmp_set) and (x.strip(' ') != 'None'):
                    conflict_tool_set.add(x.strip(' '))
                tmp_set.add(x.strip(' '))
        instruct_cnt += len(v_value[0])
        instruct_cnt += len(v_value[1])
        save_instruct[key+'+'+v_key] = len(v_value[1]) + len(v_value[0])
        save_tool[key+'+'+v_key] = len(conflict_tool_set)
        conflict_tool += len(conflict_tool_set)
        if (len(v_value[0]) + len(v_value[1])) not in instruct_dic:
            instruct_dic[(len(v_value[0]) + len(v_value[1]))] = 1
        else:
            instruct_dic[(len(v_value[0]) + len(v_value[1]))] += 1
        if (len(conflict_tool_set)) not in tool_dic:
            tool_dic[len(conflict_tool_set)] = 1
        else:
            tool_dic[len(conflict_tool_set)] += 1
        tool_cnt += len(tmp_set)
        sum_cnt += 1
    # instruct_cnt += len(instruct_answers[question][idx])
    # tool_cnt += len(tmp_set)
    # sum_cnt += 1
# for key, value in pair2.items():
#     question = key.split('_')[0]
#     idx = key.split('_')[1]
#     for v_key, v_value in value.items():
#         time_cnt += sum(times[question][idx])
#         time_cnt += sum(times[v_key.split('_')[0]][v_key.split('_')[1]])
#         tmp_set = set()
#         for tmp_key, tmp_value in v_value[1].items():
#             for x in tmp_value.split(','):
#                 tmp_set.add(x.strip(' '))
#         for tmp_key, tmp_value in v_value[0].items():
#             for x in tmp_value.split(','):
#                 if (x.strip(' ') in tmp_set) and (x.strip(' ') != 'None'):
#                     conflict_tool += 1
#                 tmp_set.add(x.strip(' '))
#         instruct_cnt += len(v_value[0])
#         instruct_cnt += len(v_value[1])
#         if (len(v_value[0]) + len(v_value[1])) not in instruct_dic:
#             instruct_dic[(len(v_value[0]) + len(v_value[1]))] = 1
#         else:
#             instruct_dic[(len(v_value[0]) + len(v_value[1]))] += 1
#         tool_cnt += len(tmp_set)
#         sum_cnt += 1
    # instruct_cnt += len(instruct_answers[question][idx])
    # tool_cnt += len(tmp_set)
    # sum_cnt += 1
print(instruct_cnt / sum_cnt)
print(tool_cnt / sum_cnt)
print(time_cnt / sum_cnt)
print(conflict_tool / sum_cnt)
print(instruct_dic)
print(tool_dic)
with open('../data/setting_setup/cars_instruct_statistic.json', 'w') as f:
    json.dump(save_instruct, f, indent=4)
with open('../data/setting_setup/cars_tool_statistic.json', 'w') as f:
    json.dump(save_tool, f, indent=4)

11.818452380952381
6.205357142857143
83.23776785714286
2.056547619047619
{7: 10, 12: 43, 10: 54, 9: 39, 11: 48, 15: 21, 8: 24, 13: 31, 14: 29, 16: 11, 17: 11, 20: 2, 23: 1, 18: 8, 19: 2, 21: 2}
{3: 84, 4: 15, 2: 145, 1: 89, 0: 3}


# Final Human Analyse

In [5]:
import json

with open('../data/match_tools/pair_tools_cars.json', 'r') as f:
    results = json.load(f)
    
with open('../data/filtered_instructions/Wikihow_filtered_instructions.json', 'r') as f:
    instructions = json.load(f)
key_list = list(results.keys())
idx = -1

In [223]:
idx += 1
key_question_set = key_list[idx]
key_question = key_question_set.split('_')[0]
key_idx = key_question_set.split('_')[1]


IndexError: list index out of range

In [224]:
tmp_key_list = list(results[key_list[idx]].keys())
tmp_idx = -1

IndexError: list index out of range

In [222]:
tmp_idx += 1
pair_question_set = tmp_key_list[tmp_idx]
pair_question = pair_question_set.split('_')[0]
pair_idx = pair_question_set.split('_')[1]
print(key_question)
print('\n'.join(instructions[key_question][key_idx]))
print(results[key_question_set][pair_question_set][0])
print('=================')
print(pair_question)
print('\n'.join(instructions[pair_question][pair_idx]))
print(results[key_question_set][pair_question_set][1])

How To Fix a Flat Tire
1. Inspect the tire for holes or protruding objects to locate where the air is escaping.
2. Inflate the tire and listen for a hissing sound.
3. Remove any objects from the tire using pliers.
4. Turn the wheel until the valve is at the top and unscrew the valve cap.
5. Attach the sealant's nozzle to the valve stem and release the contents.
6. Drive your car a few feet to distribute the sealant.
{'1': 'None', '2': 'Pump', '3': 'Pliers', '4': 'Wrench', '5': 'Wrench', '6': 'None'}
How To Clean a Car Engine
1. Remove debris from the engine bay.
2. Disconnect the battery.
3. Cover any electronics or wires.
4. Apply engine degreaser from the bottom of the engine up.
5. Allow the degreaser to soak for 5 minutes.
6. Scrub off grime using a stiff-bristled brush.
7. Rinse the engine with a garden hose.
{'1': 'None', '2': 'Wrench', '3': 'None', '4': 'Spray bottle', '5': 'None', '6': 'Stiff brush', '7': 'Hose'}
